In [8]:
import numpy as np
from scipy import sparse
from scipy.sparse import linalg

def read_sparse_matrix(filename):
    """
    Read matrix data directly into a sparse matrix format without
    creating any dense matrix.

    File format:
    First line: n (dimension)
    Then lines of:  value, row_index, col_index

    Returns:
        n (int): The dimension
        A (sparse.csr_matrix): The sparse matrix in CSR format
    """
    with open(filename, "r") as f:
        # Read dimension
        n = int(f.readline().strip())

        # Lists to store COO triplets
        data = []
        rows = []
        cols = []

        # Read the nonzero entries
        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 3:
                value, row, col = map(float, parts)
                row, col = int(row), int(col)
                data.append(value)
                rows.append(row)
                cols.append(col)

        # Create sparse matrix directly in CSR format
        A = sparse.csr_matrix((data, (rows, cols)), shape=(n, n))

    return n, A

def read_vector(filename):
    """
    Read a vector from file, with format:
    First line: n (dimension)
    Next n lines: one entry per line
    """
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read vector dimension
        b = np.array([float(f.readline().strip()) for _ in range(n)], dtype=float)
    return b

def solve_linear_system(A_file, b_file):
    """
    Solve the linear system A x = b using sparse matrices throughout
    """
    print(f"Reading matrix from {A_file}...")
    n, A = read_sparse_matrix(A_file)
    print(f"Matrix size: {n} x {n}, Number of nonzeros: {A.nnz}")

    print(f"Reading vector from {b_file}...")
    b = read_vector(b_file)
    print(f"Vector size: {len(b)}")

    print("Solving the sparse system...")
    try:
        # Use a sparse solver that works with sparse matrices directly
        x = linalg.spsolve(A, b)

        # Compute the residual using sparse operations
        residual = np.linalg.norm(A.dot(x) - b, ord=np.inf)
        print(f"Solution found! Residual ||Ax - b||∞ = {residual:.2e}")

        return x, residual
    except Exception as e:
        print(f"Error during sparse solve: {e}")
        return None, None

# Example usage
if __name__ == "__main__":
    matrix_file = "a_5.txt"
    vector_file = "b_5.txt"

    x_solution, residual = solve_linear_system(matrix_file, vector_file)

    if x_solution is not None:
        print("\nFirst 10 values of solution vector x:")
        print(x_solution[:10])

Reading matrix from a_5.txt...
Matrix size: 2025 x 2025, Number of nonzeros: 14965
Reading vector from b_5.txt...
Vector size: 2025
Solving the sparse system...
Solution found! Residual ||Ax - b||∞ = 5.23e-12

First 10 values of solution vector x:
[10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
